# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [ ]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [2]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [3]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


## List all downloaded mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show RFC1213 object tree

In [ ]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [21]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


## Agent code

In [47]:
%%writefile /tmp/agent.py
#!/usr/bin/env python3

import sys
import datetime
import socket
import json

def readJson(obj):
    f = open('/tmp/logs.json')
    data = json.load(f)
    f.close
    return data[obj]

def get_revenue():
    return readJson('revenue')

def get_total_orders():
    return readJson('totalOrders')

def get_coffee():
    return readJson('coffee')

def get_tea():
    return readJson('tea')

def get_soda():
    return readJson('soda')

def get_muffin():
    return readJson('muffin')

def get_sandwich():
    return readJson('sandwich')

def get_pie():
    return readJson('pie')

def get_status():
    return readJson('status')

def get_employees():
    return ["Gabriela","Lucas","Joana"]

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv)+"\n")

    # readJson()
    if len(sys.argv) != 3:
        print("Usage: agent.py <request-type> <MIB-oid>")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request # -s é set, -g 
        if oid == ".1.3.6.1.3.1234.1.1.0":
            print(".1.3.6.1.3.1234.1.1.0")
            print("integer")
            print(get_revenue())
        elif oid == ".1.3.6.1.3.1234.1.2.0":
            print(".1.3.6.1.3.1234.1.2.0")
            print("integer")
            print(get_total_orders())
        elif oid == ".1.3.6.1.3.1234.1.3.0":
            print(".1.3.6.1.3.1234.1.3.0")
            print("integer")
            print(get_coffee())
        elif oid == ".1.3.6.1.3.1234.1.4.0":
            print(".1.3.6.1.3.1234.1.4.0")
            print("integer")
            print(get_tea())
        elif oid == ".1.3.6.1.3.1234.1.5.0":
            print(".1.3.6.1.3.1234.1.5.0")
            print("integer")
            print(get_soda())
        elif oid == ".1.3.6.1.3.1234.1.6.0":
            print(".1.3.6.1.3.1234.1.6.0")
            print("integer")
            print(get_muffin())
        elif oid == ".1.3.6.1.3.1234.1.7.0":
            print(".1.3.6.1.3.1234.1.7.0")
            print("integer")
            print(get_sandwich())
        elif oid == ".1.3.6.1.3.1234.1.8.0":
            print(".1.3.6.1.3.1234.1.8.0")
            print("integer")
            print(get_pie())
        elif oid == ".1.3.6.1.3.1234.1.9.0":
            print(".1.3.6.1.3.1234.1.9.0")
            print("string")
            print(get_status())
        elif oid == ".1.3.6.1.3.1234.1.10.0":
            print(".1.3.6.1.3.1234.1.10.0")
            print("table")
            print(get_employees())
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()



Overwriting /tmp/agent.py


In [24]:
%%bash
sudo python /tmp/agent.py -g .1.3.6.1.3.1234.1.1.0

.1.3.6.1.3.1234.1.1.0
integer
91


## Testando usando o Agente SNMP

In [48]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.4.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.5.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.6.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.7.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.8.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.9.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.10.0

SNMPv2-SMI::experimental.1234.1.1.0 = INTEGER: 395
SNMPv2-SMI::experimental.1234.1.2.0 = INTEGER: 47
SNMPv2-SMI::experimental.1234.1.3.0 = INTEGER: 9
SNMPv2-SMI::experimental.1234.1.4.0 = INTEGER: 14
SNMPv2-SMI::experimental.1234.1.5.0 = INTEGER: 3
SNMPv2-SMI::experimental.1234.1.6.0 = INTEGER: 10


SNMPv2-SMI::experimental.1234.1.7.0 = INTEGER: 11
SNMPv2-SMI::experimental.1234.1.8.0 = INTEGER: 0
SNMPv2-SMI::experimental.1234.1.9.0 = STRING: "Open"
SNMPv2-SMI::experimental.1234.1.10.0 = No Such Instance currently exists at this OID


# Nossa MIB

In [42]:
%%writefile COFFEESHOP.txt

COFFEESHOP DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE,
    Integer32, experimental
        FROM SNMPv2-SMI
    DisplayString
        FROM SNMPv2-TC
        ;

coffee MODULE-IDENTITY
     LAST-UPDATED "9803231700Z"
     ORGANIZATION "The best coffeeshop in the world"

     CONTACT-INFO
            "        Gabriela Zorzo,
                     Lucas Andreotti,
                     Joana Figueredo
             "
     DESCRIPTION
            "The MIB Module for the best coffeeshop."
    ::= { experimental 1234 }

items OBJECT IDENTIFIER ::= { coffee 1 }

revenue OBJECT-TYPE
     SYNTAX  Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
             "The total revenue of the coffeeshop."
     ::= { items 1 }


totalOrders OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of orders placed in the coffeeshop."
    ::= { items 2 }

coffee OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of coffees ordered in the coffeeshop."
    ::= { items 3 }

tea OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of teas ordered in the coffeeshop."
    ::= { items 4 }

soda OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of sodas ordered in the coffeeshop."
    ::= { items 5 }

muffin OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of muffins ordered in the coffeeshop."
    ::= { items 6 }

sandwich OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of sandwiches ordered in the coffeeshop."
    ::= { items 7 }

pie OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of pies ordered in the coffeeshop."
    ::= { items 8 }

status OBJECT-TYPE
     SYNTAX DisplayString (SIZE(0..255))
     MAX-ACCESS read-write
     STATUS current
     DESCRIPTION
        "The status of the coffeeshop (open or closed)."
    ::= { items 9 }

employeesTable OBJECT-TYPE
    SYNTAX SEQUENCE OF Employee
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "The cofeeshop employees list."
    ::= { items 10 }

employee OBJECT-TYPE
    SYNTAX Employee
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "An employee."
    INDEX { employeeId }
    ::= { employeesTable 1 }

Employee ::= SEQUENCE {
    employeeId Integer,
    employeeName DisplayString (SIZE(0..255))
}

employeeId OBJECT-TYPE
    SYNTAX Integer
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "The employee ID."
    ::= { employee 1 }

employeeName OBJECT-TYPE
    SYNTAX DisplayString (SIZE(0..255))
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "The employee name."
    ::= { employee 2 }



END


Overwriting COFFEESHOP.txt


In [ ]:
####

# #### Nossa tabela antiga
# employeesTable OBJECT-TYPE
#      SYNTAX SEQUENCE OF Employee
#      MAX-ACCESS read-only
#      STATUS current
#      DESCRIPTION
#         "The cofeeshop employees list."
#     ::= { items 10 }

# employee OBJECT-TYPE
#      SYNTAX Employee
#      MAX-ACCESS read-only
#      STATUS current
#      DESCRIPTION
#         "An employee."
#      INDEX { employeeName }
#     ::= { employeesTable 1 }

# Employee ::= SEQUENCE {
#                 employeeName DisplayString (SIZE(0..255))
#         }

# employeeName OBJECT-TYPE
#      SYNTAX DisplayString (SIZE(0..255))
#      MAX-ACCESS read-only
#      STATUS current
#      DESCRIPTION
#         "The employee name."
#     ::= { employee 1 }

# Testando Agent SNMP acessando nossa MIB

In [44]:
%%bash

snmpget -v2c -c public -M +. -m +COFFEESHOP localhost revenue.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost totalOrders.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost coffee.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost tea.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost soda.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost muffin.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost sandwich.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost pie.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost status.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost employeesTable.0

COFFEESHOP::revenue.0 = INTEGER: 226
COFFEESHOP::totalOrders.0 = INTEGER: 2
COFFEESHOP::coffee.0 = INTEGER: 3
COFFEESHOP::tea.0 = INTEGER: 6
COFFEESHOP::soda.0 = INTEGER: 4


COFFEESHOP::muffin.0 = INTEGER: 1
COFFEESHOP::sandwich.0 = INTEGER: 2
COFFEESHOP::pie.0 = INTEGER: 1
COFFEESHOP::status.0 = STRING: Open
COFFEESHOP::employeesTable.0 = No Such Instance currently exists at this OID


# Nossa Aplicação que gera os logs

In [19]:
%%writefile /tmp/aplicacao.py

from random import randrange
import time
import json

# Funcionários
employees = ["Gabriela","Joana","Lucas"]

# Items
items = ["coffee","tea","soda","muffin","sandwich","pie"]

# Número items
itemsNumber = [0,0,0,0,0,0]

# Tabela de preços
priceTable = {"coffee" : 8, "tea" : 5, "soda" : 6, "muffin" : 7, "sandwich" : 15, "pie" : 12 }

# Número pedidos
orders = 0

# Total valor
totalValue = 0

# Status
status = "Open"


while(True):
    index = randrange(0,5)
    item = items[index]
    value = priceTable[item]

    orders += 1
    totalValue += value
    itemsNumber[index] += 1

    log = {
        "revenue" : totalValue,
        "totalOrders" : orders,
        items[0] : itemsNumber[0],
        items[1] : itemsNumber[1],
        items[2] : itemsNumber[2],
        items[3] : itemsNumber[3],
        items[4] : itemsNumber[4],
        items[5] : itemsNumber[5],
        "employees" : employees,
        "priceTable" : priceTable,
        "status" : status
    }

    print(log)
   
    json_object = json.dumps(log, indent=4)
    
    with open("/tmp/logs.json", "w") as outfile:
        outfile.write(json_object)
        outfile.close()

    time.sleep(10)

Overwriting /tmp/aplicacao.py
